# I. Import Libraries:

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [ ]:
import tensorflow as tf
print(tf.__version__)

# II. Edit Data:

In [93]:
DATA_ROOT = '/home/jupyter/lung_x_ray/data/'
PROJECT_FOLDER = '/home/jupyter/lung_x_ray'
MODEL_FOLDER = '/home/jupyter/lung_x_ray/trained_model/xception'

In [94]:
train = pd.read_csv('/home/jupyter/lung_x_ray/data/CheXpert-v1.0-small/Final_list/train_final.csv')
valid = pd.read_csv('/home/jupyter/lung_x_ray/data/CheXpert-v1.0-small/Final_list/valid_final.csv')

train.sample(5)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_paths
30254,CheXpert-v1.0-small/train/patient09374/study7/...,Female,53,Frontal,AP,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,/home/jupyter/lung_x_ray/data/CheXpert-v1.0-sm...
31349,CheXpert-v1.0-small/train/patient09717/study1/...,Male,73,Frontal,AP,NaN,NaN,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,/home/jupyter/lung_x_ray/data/CheXpert-v1.0-sm...
151067,CheXpert-v1.0-small/train/patient42835/study8/...,Male,58,Frontal,AP,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,/home/jupyter/lung_x_ray/data/CheXpert-v1.0-sm...
19576,CheXpert-v1.0-small/train/patient06051/study9/...,Male,71,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,/home/jupyter/lung_x_ray/data/CheXpert-v1.0-sm...
197088,CheXpert-v1.0-small/train/patient07123/study3/...,Male,55,Lateral,NaN,NaN,NaN,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,/home/jupyter/lung_x_ray/data/CheXpert-v1.0-sm...


In [95]:
train = train[train['No Finding'] != 1.0]
valid = valid[valid['No Finding'] != 1.0]

In [98]:
train = train[train['Frontal/Lateral'] == 'Lateral']
valid = valid[valid['Frontal/Lateral'] == 'Lateral']

In [100]:
train = train.rename(columns={"img_paths": "image_paths"})
valid = valid.rename(columns={"img_paths": "image_paths"})

- Drop all un-use columns

In [102]:
train.drop(['Path','Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding'], axis=1, inplace = True)
valid.drop(['Path','Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding'], axis=1, inplace = True)

## 1. Label Processing:
### a. Change all NaN (Blank) => 0 same as negative



In [104]:
train.fillna(value=0, inplace=True)
valid.fillna(value=0, inplace=True)

### b. Apply U-One approach => Change all Uncertain (-1) to random number in range 0.55 - 0.85

In [106]:
import random

for i in range (0,13):
    train.iloc[: , i] = train.iloc[:, i].map(lambda x: random.uniform(0.55, 0.85) if x == -1 else x)
    valid.iloc[: , i] = valid.iloc[:, i].map(lambda x: random.uniform(0.55, 0.85) if x == -1 else x)

## 2. Split train and valid data:

In [108]:
train_amount = int(len(list(train['image_paths']))*0.8)
data_set_change = train.iloc[train_amount:,:]
train = train.drop(train.index[train_amount:], axis=0)
valid = pd.concat([valid, data_set_change])

In [ ]:
train_img = train['image_paths']
val_img = valid['image_paths']

## 3. Generate dataset:

In [117]:
label_col = list(train.columns)[:-1]

label_col

['Enlarged Cardiomediastinum',
 'Cardiomegaly',
 'Lung Opacity',
 'Lung Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural Effusion',
 'Pleural Other',
 'Fracture',
 'Support Devices']

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=15,
    zoom_range=[0.95, 1.05],
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.15,
    fill_mode="constant",
    cval=128
)

train_ds = image_generator.flow_from_dataframe(train,
                                                color_mode='rgb',
                                                 x_col='image_paths',
                                                 y_col=label_col,
                                                 class_mode='raw',
                                                 target_size=(299,299),
                                                 shuffle=True,
                                                 batch_size=32)

val_ds = image_generator.flow_from_dataframe(valid,
                                                    color_mode='rgb',
                                                    x_col='image_paths',
                                                    y_col=label_col,
                                                    class_mode='raw',
                                                    target_size=(299,299),
                                                    batch_size=32)

# III. Build Model:

In [121]:
inception = tf.keras.applications.InceptionResNetV2(input_shape=(299, 299, 3), weights="imagenet", include_top=False)

In [ ]:
inception.trainable=True

In [124]:
fine_tune_at = 385

In [125]:
for layer in inception.layers[:fine_tune_at]:
    layer.trainable = False

In [126]:
model2 = tf.keras.Sequential([
    inception,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(13, activation = 'sigmoid')])

In [127]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2=0.999),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model2.summary()

# IV. Training:

In [120]:
#Setting up CheckPoint 
checkpoint_path = os.path.join(MODEL_FOLDER, "lateral_inception_ckpt.h5")
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# by default it saves the weights every epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only = True,
                                                 save_weights_only=True,
                                                 mornitor = 'val_auc',
                                                 verbose=1)

In [ ]:
TRAIN_SAMPLES = len(train_img)
VAL_SAMPLES = len(val_img)

In [ ]:
# Train the model with Train data
steps_per_epoch = len(train_img) // 32
validation_steps = len(val_img) // 32

history = model2.fit(train_ds, 
                    epochs=7,
                    steps_per_epoch = steps_per_epoch,
                    validation_data= val_ds,
                    validation_steps = validation_steps,
                   callbacks=[cp_callback])

In [92]:
model2.save('./trained_model/lateral_inception.h5')